In [ ]:
from datasets import Dataset, load_dataset
task_dataset = load_dataset("json", data_files="./datasets/hellaswag/train.json")["train"]
new_splits = task_dataset.train_test_split(test_size=4000)
new_splits

In [ ]:
train_dataset, dev_dataset = new_splits["train"], new_splits["test"]
train_dataset.to_json("./datasets/hellaswag_36k/train.json")
dev_dataset.to_json("./datasets/hellaswag_4k/train.json")